In [1]:
from flask import Flask, render_template
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
import re
import requests

url = "https://data.cityofchicago.org/resource/qzdf-xmn8.json"
response = requests.get(url)
data = response.json()

# Convert to DataFrame to inspect the structure
df = pd.DataFrame(data)
print(df.head())
print(df.columns)

                      date                                           location  \
0  2020-12-31T23:59:00.000  {'latitude': '41.9116839', 'human_address': '{...   
1  2020-12-31T23:51:00.000  {'latitude': '41.863406877', 'human_address': ...   
2  2020-12-31T23:45:00.000  {'latitude': '41.792043078', 'human_address': ...   
3  2020-12-31T23:45:00.000  {'latitude': '41.947024331', 'human_address': ...   
4  2020-12-31T23:45:00.000  {'latitude': '41.772893757', 'human_address': ...   

  district                  block y_coordinate      latitude  \
0      025     017XX N CICERO AVE      1910955    41.9116839   
1      010      013XX S HOMAN AVE      1893431  41.863406877   
2      002     056XX S WABASH AVE      1867609  41.792043078   
3      016  048XX W PATTERSON AVE      1923829  41.947024331   
4      003   002XX E MARQUETTE RD      1860641  41.772893757   

                           description  \
0                            OVER $500   
1  AGGRAVATED - OTHER DANGEROUS WEAPON   
2 

In [2]:
df['Latitude'] = df['location'].apply(lambda x: x.get('latitude') if pd.notnull(x) else None)
df['Longitude'] = df['location'].apply(lambda x: x.get('longitude') if pd.notnull(x) else None)


In [8]:
crime = pd.DataFrame(data)

# Assuming 'latitude' and 'longitude' are direct columns in the DataFrame
# Replace null or whitespace strings in latitude and longitude columns
crime['latitude'] = crime['latitude'].replace(r'\s+', np.nan, regex=True)
crime['latitude'] = crime['latitude'].replace(r'^$', np.nan, regex=True)
crime['longitude'] = crime['longitude'].replace(r'\s+', np.nan, regex=True)
crime['longitude'] = crime['longitude'].replace(r'^$', np.nan, regex=True)

# Convert latitude and longitude to numeric, filling NaNs with a placeholder
crime['latitude'] = pd.to_numeric(crime['latitude'], errors='coerce').fillna(-0.99999)
crime['longitude'] = pd.to_numeric(crime['longitude'], errors='coerce').fillna(-0.99999)

# Drop unnecessary columns, assuming these are the region columns that you don't need
columns_to_drop = [':@computed_region_b3wi_w8ix', ':@computed_region_fhmw_rucx', ':@computed_region_u3y2_d2ws',
                   ':@computed_region_5s6d_2f32', ':@computed_region_3ini_iehf', ':@computed_region_5bih_7r3y',
                   ':@computed_region_x3q3_gi3e']
crime.drop(columns=columns_to_drop, errors='ignore', inplace=True)

# Drop any rows that still contain NaN values
crime.dropna(inplace=True)

# Create the map centered around a point in Chicago
map = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

# Prepare data for the map
lat = crime['latitude'].tolist()
lng = crime['longitude'].tolist()
offense = crime['primary_type'].tolist()  # Assuming 'primary_type' as the crime type column
locations = list(zip(lat, lng))

# Create a MarkerCluster object
marker_cluster = MarkerCluster(name="Crimes by Marker", overlay=True, control=True)

# Loop through the data and add each marker to the cluster
for i in range(len(locations)):
    location = locations[i]
    crime_type = offense[i]
    html = f'''<b>Type of Crime:</b> {crime_type}<br>Latitude: {location[0]}<br>Longitude: {location[1]}'''
    iframe = folium.IFrame(html, width=200, height=200)
    popup = folium.Popup(iframe, max_width=200)
    marker = folium.Marker(location=location, popup=popup)
    marker_cluster.add_child(marker)

# Add the marker cluster to the map
marker_cluster.add_to(map)

# Add a heatmap to the map
HeatMap(data=locations, name="Crimes by Heatmap").add_to(map)
folium.LayerControl().add_to(map)

# Generate the map's HTML representation
html_map = map._repr_html_()

# Setup the Flask application
app = Flask(__name__)

@app.route("/")
def index():
    return render_template("index.html", map=html_map)

@app.route("/map/")
def map():
    return render_template("map.html")

@app.route("/contact/")
def contact():
    return render_template("contact.html")

@app.route("/avoid/")
def avoid():
    return render_template("avoidance_map.html")

if __name__ == "__main__":
    app.run(host="127.0.0.1", port=8000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/shafwatmustafa/Documents/DS440/crime-app-main/flaskenv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/shafwatmustafa/Documents/DS440/crime-app-main/flaskenv/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/Users/shafwatmustafa/Documents/DS440/crime-app-main/flaskenv/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/Users/shafwatmustafa/

SystemExit: 1

/Users/shafwatmustafa/Documents/DS440/crime-app-main/flaskenv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 